In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
import dropbox
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

**To transfer data from sun to dropbox**
- select the path where the data is


In [ ]:
directory_origin = "/mnt/sun-temp/TEMP/PRINCE_Felix/"
directory_origin = "/media/bisot/AMF_03/PRINCE/"

- get the folders at this path and select the folders of interest

In [ ]:
update_plate_info(directory_origin, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_origin, local=True)
folders = all_folders

- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go
- The delete parameter indicates wether the data should be deleted from the path of origin after succesfull upload on dropbox

In [ ]:
dir_drop = "DATA/PRINCE"
delete = False
run_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
)
clear_output(wait=False)

**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [ ]:
all_folders_drop = get_dropbox_folders("/DATA/TransportCUT/", True)

In [ ]:
# folders_drop = all_folders_drop.loc[all_folders_drop['CrossDate'].between("20181015","20201015")]
folders_drop = all_folders_drop.loc[all_folders_drop["Plate"] == "52"]

In [ ]:
folders_drop

- select the path where the folders should go

In [ ]:
# directory_targ = '/mnt/sun-temp/TEMP/PRINCE_Felix/'
directory_targ = "/mnt/sun/home-folder/bisot/PRINCE_test/"

- use the 'from_drop.py' function including the folders of interest

In [ ]:
run_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
)
clear_output(wait=False)

**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [ ]:
folders = all_folders.loc[all_folders["Plate"].isin([792, 94])]

In [ ]:
len(folders)

In [ ]:
unzip = "False"
flatten = False
dir_drop = "prince_data"

directory_targ = os.path.join(directory_scratch, "temp") + "/"
directory = directory_targ
run_parallel_transfer(
    "from_drop.py",
    [directory_targ, dir_drop, unzip, flatten],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="archiving.sh",
)
run_parallel_transfer_to_archive(
    folders, directory, "2:00:00", "staging", name_job="archiving.sh"
)

In [ ]:
run_parallel_transfer_to_archive(
    folders, directory, "10:00", "staging", name_job="archiving_no_wait.sh"
)

In [ ]:
must_zip = True
must_unzip = False
run_parallel_transfer(
    "within_surf.py",
    [directory, directory_targ, must_zip, must_unzip],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
)

In [ ]:
def get_dropbox_folders_new(dir_drop: str) -> pd.DataFrame:
    data = []
    dbx = load_dbx()
    response = dbx.files_list_folder(dir_drop, recursive=True)
    # for fil in response.entries:
    listfiles = []
    listjson = []
    while response.has_more:
        listfiles += [
            file for file in response.entries if file.name.split("/")[-1] == "param.m"
        ]
        response = dbx.files_list_folder_continue(response.cursor)
    listfiles += [
        file for file in response.entries if file.name.split("/")[-1] == "param.m"
    ]
    # print([((file.path_lower.split(".")[0]) + "_info.json") for file in listfiles if (file.name.split(".")[-1] == "zip") &
    #        (((file.path_lower.split(".")[0]) + "_info.json") not in listjson)])
    listfiles.reverse()
    names = [file.path_lower.split("/")[-2] for file in listfiles]
    path_drop = [os.path.join(*file.path_lower.split("/")[:-1]) for file in listfiles]
    id_uniques = [path.split("/")[-2] for path in path_drop]
    plate_num = [idi.split("_")[0] for idi in id_uniques]
    date_cross = [idi.split("_")[1] for idi in id_uniques]
    sizes = [file.size / 10**9 for file in listfiles]
    modified = [file.client_modified for file in listfiles]
    df = pd.DataFrame(
        (names, sizes, modified, path_drop, plate_num, date_cross, id_uniques)
    ).transpose()
    df = df.rename(
        columns={
            0: "folder",
            1: "size",
            2: "change_date",
            3: "tot_path_drop",
            4: "Plate",
            5: "CrossDate",
            6: "unique_id",
        }
    )
    return df

In [ ]:
all_folders_drop_new = get_dropbox_folders_new("/DATA/PRINCE")

In [ ]:
all_folders_drop_new

In [ ]:
directory_targ2 = os.path.join(directory_scratch, "temptemp") + "/"

In [ ]:
dbx = load_dbx()

path = "/" + all_folders_drop_new.iloc[0]["tot_path_drop"]
response = dbx.files_list_folder(path, recursive=False)
listfiles = []
listjson = []
while response.has_more:
    listfiles += [file for file in response.entries]
    response = dbx.files_list_folder_continue(response.cursor)
listfiles += [file for file in response.entries]

In [ ]:
listfiles

In [ ]:
[file.path_lower for file in listfiles]

In [ ]:
def download_folders_drop(folders_drop: pd.DataFrame, directory_target):
    dbx = load_dbx()
    for index, row in folders_drop.iterrows():
        path = "/" + row["tot_path_drop"]
        response = dbx.files_list_folder(path, recursive=False)
        listfiles = []
        while response.has_more:
            listfiles += [file for file in response.entries]
            response = dbx.files_list_folder_continue(response.cursor)
        listfiles += [file for file in response.entries]
        folder = row["folder"]
        path_folder = os.path.join(directory_target, folder)
        if not os.path.exists(path_folder):
            os.mkdir(path_folder)
        for file in listfiles:
            path_drop = file.path_lower
            path_local = os.path.join(
                directory_target, folder, path_drop.split("/")[-1]
            )
            print(path_drop, path_local)

            download(path_drop, path_local, unzip=(path_drop[-4:] == ".zip"))

In [ ]:
download_folders_drop(all_folders_drop_new, directory_targ2)